## Simple Functorch but Make it Modes

This notebook is a rewrite of the simple functorch notebook that uses torch dispatch modes instead of the Dispatcher object

In [ ]:
!pip uninstall -y torch
!pip install --pre torch -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html --upgrade

Found existing installation: torch 1.13.0.dev20220613+cpu
Uninstalling torch-1.13.0.dev20220613+cpu:
  Successfully uninstalled torch-1.13.0.dev20220613+cpu
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
  Using cached https://download.pytorch.org/whl/nightly/cpu/torch-1.13.0.dev20220613%2Bcpu-cp37-cp37m-linux_x86_64.whl (190.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.13.0.dev20220613+cpu which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.13.0.dev20220613+cpu which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.13.0.dev20220613+cpu which is incompatible.


This notebook walks through a self-contained implementation of
functorch, including support for both vjp and vmap combinators (using
PyTorch only to implement primitive tensor operations).  It follows
the tradition of
[Autodidax](https://jax.readthedocs.io/en/latest/autodidax.html) (a
pedagogical reimplementation of JAX, the library functorch is inspired
by) and [Simple
Autograd](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC?usp=sharing)
(Zachary Devito's pedagogical reimplementation of autograd, which the
autograd system in this notebook is based off of.) You can [open this
file in
Colab](https://colab.research.google.com/github/albanD/subclass_zoo/blob/main/simple_functorch.ipynb)
and play around with the examples.

As a simplified implementation of functorch, this notebook also makes
it easier to investigate some more subtle aspects of how PyTorch's
native autograd system interacts with composable transforms.  In
particular, we will see that PyTorch's native implementation of double
backwards (which shares the same tape through multiple levels of
differentiation) differs from functorch's nested grad implementation
(which maintains a separate tape per level).

To get started, we replicate some of the data structures and helper functions
from Simple Autograd.

In [ ]:
import contextlib
import functools
from dataclasses import dataclass
from typing import Callable, Dict, List, NamedTuple, Optional

import torch
from torch import Tensor
from torch.utils._python_dispatch import TorchDispatchMode


class TapeEntry(NamedTuple):
    # names of the inputs to the original computation
    inputs: List[str]
    # names of the outputs of the original computation
    outputs: List[str]
    # apply chain rule
    propagate: Callable[[List[Tensor]], List[Tensor]]


_name = 0


def fresh_name() -> str:
    """create a new unique name for a variable: v0, v1, v2"""
    global _name
    r = f"v{_name}"
    _name += 1
    return r

This is a little helper function for converting the dim argument in
sum into an explicit list of dimensions that will be reduced over.
It takes the dim of the tensor we are summing over and the dim
argument itself.

In [ ]:
def sum_dims(*, input_dim, dim):
    if dim is None:
        return tuple(range(0, input_dim))
    elif isinstance(dim, int):
        return (dim,)
    else:
        return tuple(sorted(dim))

This is another little helper function that we might want to incorporate into the default behavior of restore. But for now we need this in order to not error when restoring a mode

In [ ]:
from torch._C import _get_torch_dispatch_mode

def restore_dispatcher_or_nop(dispatcher):
    # another argument that maybe we should let .restore() keep the current mode
    if dispatcher == _get_torch_dispatch_mode():
        # we'll just no-op here since restoring the current mode will error
        return contextlib.nullcontext
    return dispatcher.restore

In Simple Autograd, we provided a Variable wrapper class which
provided a traditional Tensor style interface for our objects; in
functorch proper, objects are repeatedly wrapped in this way to
implement multipler layers of transformations.

In my opinion, this sort of wrapper makes it more difficult to
understand the flow of logic.  So in Simple Functorch, we take a
different approach: we won't make use of a wrapper class at all,
instead showing how to add it in the end as syntax sugar on top of our
system.

For debuggability purposes, however, it is nice to have a way to
identify variables by a human readable name.  We'll do this by setting
a t_name attribute on PyTorch tensors whenever we allocate a new
tensor.

In [ ]:
def label(t: Tensor, name: str = None):
    if not hasattr(t, "t_name"):
        t.t_name = name or fresh_name()
    return t

So if we aren't going to have a wrapper around each tensor, how will
we actually implement our logic?  We will organize our various layers
of transformations as separate Dispatcher objects, which inherit from mode and define methods for performing operations on tensors, but are not Tensors
themselves. For example, instead of defining Tensor.add(Tensor), the mode will catch the add(Tensor, Tensor) call when it hits the Pytorch dispatcher. In order to avoid the same boilerplate in every dispatcher object, we define a parent object that catches all functions and redispatches it to the correct rule based on the child Dispatcher's implementation

Notice that unlike with the original simple functorch, we don't have to set an
inner parameter. This logic is handeled by the underlying mode implementation

In [ ]:
class Dispatcher(TorchDispatchMode):
    def apply(self, func):
        if func.__name__ == "add.Tensor":
            return self.add
        if func.__name__ == "mul.Tensor":
            return self.mul
        if func.__name__ in ["sum.default", "sum.dim_IntList"]:
            return self.sum
        if func.__name__ == "expand.default":
            return self.expand
        if func.__name__ == "unsqueeze.default":
            return self.unsqueeze
        if func.__name__ == "squeeze.dim":
            return self.squeeze
        if func.__name__ == "size":
            return self.size
        if func.__name__ == "ones.default":
            return self.ones
        else:
            raise RuntimeError(f"Simple functorch doesn't support {func.__name__}")

    def mul(self, lhs, rhs):
        raise NotImplementedError

    def add(self, lhs, rhs):
        raise NotImplementedError

    # Sum has been generalized to take an optional dim argument, which we
    # will need for Batched tensors
    def sum(self, input, dim=None):
        raise NotImplementedError

    def expand(self, input, sizes):
        raise NotImplementedError

    # For closure under Batched tensors, we need these operations...
    def unsqueeze(self, input, dim):
        raise NotImplementedError

    def squeeze(self, input, dim):
        raise NotImplementedError

    # ...and we also need to overload the meaning of size/ones to
    # hide/reinsert batch dimensions.  We also introduce a concept
    # of "lifting" a tensor to be batched by broadcasting it on
    # a dimension
    def size(self, input):
        raise NotImplementedError

    def ones(self, size, **kwargs):
        raise NotImplementedError

    def lift(self, input, d):
        raise NotImplementedError

    # For convenience, we provide dim, which just returns the length of
    # the sizes
    def dim(self, input):
        return len(self.size(input))
    
    def custom_vjp(self, fwd_fn, bwd_fn, *args):
        # really gross but because we don't have the torch_dispatch for this, we
        # need something to mimic what the mode does in torch_dispatch
        old = torch._C._get_torch_dispatch_mode()
        try:
            torch._C._set_torch_dispatch_mode(None)  # BUG: should be able to be done with enable
            with self.inner.restore():
              return self.inner.custom_vjp(fwd_fn, bwd_fn, *args)
        finally:
          torch._C._set_torch_dispatch_mode(old)

    def __torch_dispatch__(self, func, types, args=(), kwargs=None):
        kwargs = kwargs if kwargs else {}
        return self.apply(func)(*args, **kwargs)

To start with, we can implement a labeler layer, which just labels all inputs
and outputs. This will be necessary for autograd so it should be the bottom
most layer to everything. Specifically, we'll set it's inner to be None so that
if it's used as not the innermost layer, it will error

In [ ]:
class Labeler(Dispatcher):
    def mul(self, lhs, rhs):
        return label(torch.mul(lhs, rhs))

    def add(self, lhs, rhs):
        return label(torch.add(lhs, rhs))

    def sum(self, input, dim=None):
        if dim is None:
            return label(torch.sum(input))
        else:
            return label(torch.sum(input, dim))

    def expand(self, input, sizes):
        return label(input.expand(sizes))

    def unsqueeze(self, input, dim):
        return label(torch.unsqueeze(input, dim))

    def squeeze(self, input, dim):
        return label(torch.squeeze(input, dim))

    def size(self, input, **kwargs):
        # Return size a tuple for marginally more compact printing
        assert isinstance(input, torch.Tensor)
        return input.size()

    def ones(self, size, **kwargs):
        return label(torch.ones(size))
    
    def lift(self, input, d):
        assert self == d
        return input

    def custom_vjp(self, fwd_fn, bwd_fn, *args):
        # The backend layer for custom_vjp just calls fwd_fn.
        # Why doesn't it create an autograd.Function? We're assuming the backend
        # layer doesn't need to handle Autograd.
        assert self.inner == None
        a, b = fwd_fn(*args)
        result = label(a), label(b)
        return result

Dispatcher layers are composable via object composition: we can
imagine a stack of dispatchers, each one calling into the next.
For example, the Logger dispatcher simply prints out what operation
was called on it, and then forwards on the operation to the inner
dispatcher. Unlike with simple functorch, we're able to rely on the modes to forward the call to the inner dispatcher by just calling the function again

In [ ]:
def custom_vjp_str(r, fwd_fn, bwd_fn, args):
    arg_names = ", ".join([a.t_name for a in args])
    r_is_tensor = isinstance(r, torch.Tensor)
    if r_is_tensor:
        result_names = r.t_name
    else:
        result_names = [r.t_name for r in r]
        if len(result_names) == 1:
            result_names = f"{result_names[0]},"
        else:
            result_names = ", ".join(result_names)

    print(
        f"{result_names} = custom_vjp({fwd_fn.__name__}, {bwd_fn.__name__}, {arg_names})"
    )

In [ ]:
class Logger(Dispatcher):
    def __init__(self, *, name):
        self.name = f"  {name}"

    def size(self, input):
        # don't log size calls
        return self.inner.size(input)

    def ones(self, size, **kwargs):
        r = torch.ones(size)
        print(f"{self.name} {r.t_name}: {self.size(r)} = ones({size})")
        return r

    def mul(self, lhs, rhs):
        r = lhs.mul(rhs)
        if isinstance(rhs, float):
            print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs}")
        else:
            print(
                f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs.t_name}"
            )
        return r

    def add(self, lhs, rhs):
        r = lhs.add(rhs)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} + {rhs.t_name}")
        return r

    def sum(self, input, dim=None):
        if dim is None:
          r = input.sum()
        else:
          r = input.sum(dim)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.sum(dim={dim})")
        return r

    def unsqueeze(self, input, dim):
        r = input.unsqueeze(dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.unsqueeze({dim})"
        )
        return r

    def squeeze(self, input, dim):
        r = input.squeeze(dim)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.squeeze({dim})")
        return r

    def expand(self, input, sizes):
        r = input.expand(sizes)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.expand({sizes})"
        )
        return r

    def custom_vjp(self, fwd_fn, bwd_fn, *args):
        # because custom_vjp is not an aten function, we have to explicitly send
        # it to its inner 
        r = super().custom_vjp(fwd_fn, bwd_fn, *args)
        print(custom_vjp_str(r, fwd_fn, bwd_fn, args))
        return r
    
    def lift(self, input, d):
        if self == d:
            return input
        else:
            return self.inner.lift(input, d)

Here is a simple example of using Logger and Torch together.  Whenever
we make calls to operations, we must do so via the Dispatcher object.
We will explicitly write out all of these calls before we add wrapper
class sugaring.

In [ ]:
with Labeler():
    with Logger(name="Torch"):
      z = torch.ones(2) + torch.ones(2)
print(z)
assert(isinstance(z, torch.Tensor))

  Torch v338: torch.Size([2]) = ones([2])
  Torch v339: torch.Size([2]) = ones([2])
  Torch v340: torch.Size([2]) = v338 + v339
tensor([2., 2.])


With the Dispatcher structure in hand, we are now in a good place to
port the autograd implementation from Simple Autograd into our new
framework.

In [ ]:
from torch.utils._mode_utils import no_dispatch

class Autograd(Dispatcher):
    # create_graph here corresponds to the create_graph kwarg in traditional
    # PyTorch, which controls whether or not the graph of the derivative
    # will be constructed, allowing computing higher order derivatives.
    # We will see that although create_graph=True allows Autograd to directly
    # support higher order derivatives, layering an Autograd to another
    # Autograd will also allow higher order derivatives.
    def __init__(self, *, name="Autograd", create_graph: bool = False):
        self.gradient_tape = []
        self.name = name
        self.create_graph = create_graph

    # create_graph controls where add/mul/etc calls from the backwards
    # propagators go: if you create_graph, they we're going to have you
    # the current Autograd dispatcher; otherwise they're going to
    # move on to the inner layer. This restores the right mode to reset (and
    # the proper context manager to use)
    def backward_inner(self):
        if self.create_graph:
            mode = self
        else:
            mode = self.inner
        return restore_dispatcher_or_nop(mode)

    def mul(self, lhs, rhs):
        if isinstance(rhs, float) and rhs == 1.0:
            # peephole optimization
            return lhs

        # define forward
        # first, run the operation in the inner layer to get the initial
        # result
        r = lhs.mul(rhs)
        # We directly implement printing here as it indicates whether or not
        # this operation was saved to the tape or not
        if isinstance(rhs, float):
            print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs}")
        else:
            print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} * {rhs.t_name}")

        # record what the inputs and outputs of the op were
        inputs = [lhs.t_name, rhs] if isinstance(rhs, float) else [lhs.t_name, rhs.t_name]
        outputs = [r.t_name]

        # define backprop
        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs

            dr_dlhs = rhs  # partial derivative of r = lhs*rhs
            dr_drhs = lhs  # partial derivative of r = lhs*rhs

            # chain rule propagation from outputs to inputs of multiply.
            # Notice that the propagation rule may itself call
            # other operations; depending on create_graph, they may
            # either be dispatched with self or self.inner; self.backward_inner()
            # controls which one we go to.
            with self.backward_inner()():
                dL_dlhs = dL_dr.mul(dr_dlhs)
                dL_drhs = dL_dr.mul(dr_drhs)
            dL_dinputs = [dL_dlhs, dL_drhs]
            return dL_dinputs

        # finally, we record the compute we did on the tape
        self.gradient_tape.append(
            TapeEntry(inputs=inputs, outputs=outputs, propagate=propagate)
        )
        return r

    # The rest of the implementations follow in the same way and can
    # be skipped

    def add(self, lhs, rhs):
        # Add follows a similar pattern to Mul, but it doesn't end up
        # capturing any variables.
        r = lhs.add(rhs)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {lhs.t_name} + {rhs.t_name}")

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            dr_dlhs = 1.0
            dr_drhs = 1.0
            with self.backward_inner()():
                dL_dlhs = dL_dr.mul(dr_dlhs)
                dL_drhs = dL_dr.mul(dr_drhs)
            return [dL_dlhs, dL_drhs]

        self.gradient_tape.append(
            TapeEntry(
                inputs=[lhs.t_name, rhs.t_name], outputs=[r.t_name], propagate=propagate
            )
        )
        return r

    # Extended to handle dim argument for Batched (later)
    def sum(self, input: Tensor, dim=None):
        if dim is None:
          r = input.sum()
        else:
          r = input.sum(dim)
        print(f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.sum(dim={dim})")

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            size = self.size(input)
            res = dL_dr
            # Broadcast over all dimensions that were reduced over
            input_dim = self.inner.dim(input)  # this needs to be done in inner
            print(input_dim)
            print(dim)
            with self.backward_inner()():
                for i in sum_dims(input_dim=input_dim, dim=dim):
                    res = res.unsqueeze(i)
                out = res.expand(size)
            return [out]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    # Unlike Simple Autograd, this expand requires the input to have
    # been unsqueezed before hand.  This lets us avoid having to do
    # at::sum_to for the nontrivial case (which is more complicated)
    def expand(self, input: Tensor, sizes: List[int]):
        print(self.inner.dim(input))
        print(len(sizes))
        assert self.inner.dim(input) == len(sizes)  # only works if dims match
        r = input.expand(sizes)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.expand({sizes})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            input_size = input.size()
            dims = tuple(
                i for i in range(input.dim()) if input_size[i] != sizes[i]
            )
            # We wanted a sum keepdim=True, but I didn't want to force
            # everyone to support it so manually unsqueeze
            with self.backward_inner()():
                res = dL_dr.sum(dims)
                for d in dims:
                    res = res.unsqueeze(d)
            return [res]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    def squeeze(self, input: Tensor, dim):
        r = input.squeeze(dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.squeeze(dim={dim})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_outputs
            with self.backward_inner()():
                res = dL_dr.unsqueeze(dim)
            return [res]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    def unsqueeze(self, input: Tensor, dim):
        r = input.unsqueeze(dim)
        print(
            f"{self.name} {r.t_name}: {self.size(r)} = {input.t_name}.unsqueeze(dim={dim})"
        )

        def propagate(dL_doutputs: List[Tensor]):
            (dL_dr,) = dL_doutputs
            with self.backward_inner()():
                out = dL_dr.squeeze(dim)
            return [out]

        self.gradient_tape.append(
            TapeEntry(inputs=[input.t_name], outputs=[r.t_name], propagate=propagate)
        )
        return r

    def ones(self, size, **kwargs):
        return torch.ones(size)
    
    def custom_vjp(self, fwd_fn, bwd_fn, *args):
      # To support Autograd(Autograd(Torch()), custom_vjp MUST call custom_vjp
      # on the inner dispatcher. If it instead called fwd_fn(*args), then
      # the inner Autograd dispatcher would not use bwd_fn in its backward pass.

      r, saved = super().custom_vjp(fwd_fn, bwd_fn, *args)
      print(custom_vjp_str(r, fwd_fn, bwd_fn, args))

      # To preserve custom backward semantics, we create a lambda that calls
      # bwd_fn. This lambda is then saved on the gradient tape.
      def propagate(dL_doutputs: List[Tensor]):
          with self.backward_inner()():
              return bwd_fn(dL_doutputs, saved)

      self.gradient_tape.append(
          TapeEntry(
              inputs=[arg.t_name for arg in args], outputs=[r.t_name], propagate=propagate
          )
      )
      return r, saved

    def size(self, input):
        return self.inner.size(input)
    
    def lift(self, input, d):
        if self == d:
            return input
        else:
            return self.inner.lift(input, d)

    def grad(self, L, desired_results: List[Tensor]) -> List[Tensor]:
        # this map holds dL/dX for all values X
        dL_d: Dict[str, Tensor] = {}
        # It starts by initializing the 'seed' dL/dL, which is 1
        # TODO: indirect this via the backend
        with restore_dispatcher_or_nop(self.inner)():
            dL_d[L.t_name] = torch.ones(self.inner.size(L))  
        print(f"-- {self.name} d{L.t_name} -------")

        # look up dL_dentries. If a variable is never used to compute the loss,
        # we consider its gradient None, see the note below about zeros for more information.
        def gather_grad(entries: List[str]):
            return [dL_d[entry] if entry in dL_d else None for entry in entries]

        # propagate the gradient information backward
        for entry in reversed(self.gradient_tape):
            dL_doutputs = gather_grad(entry.outputs)
            if all(dL_doutput is None for dL_doutput in dL_doutputs):
                # optimize for the case where some gradient pathways are zero. See
                # The note below for more details.
                continue

            # perform chain rule propagation specific to each compute
            dL_dinputs = entry.propagate(dL_doutputs)

            # Accululate the gradient produced for each input.
            # Each use of a variable produces some gradient dL_dinput for that
            # use. The multivariate chain rule tells us it is safe to sum
            # all the contributions together.
            for input, dL_dinput in zip(entry.inputs, dL_dinputs):
                if input not in dL_d:
                    dL_d[input] = dL_dinput
                else:
                    with self.backward_inner()():
                      dL_d[input] = dL_d[input].add(dL_dinput)

        # print some information to understand the values of each intermediate
        # for name, value in dL_d.items():
        #     print(f'{self.name} d{L.t_name}_d{name} = {value.t_name}')
        print(f"------------------------")

        return gather_grad(desired.t_name for desired in desired_results)

To calculate some simple gradients, we can compose Autograd with
Torch and get the result we expect.

In [ ]:
torch.manual_seed(0)
a, b = label(torch.rand(4)), label(torch.rand(4))

def simple(a, b):
    t = a + b
    return t.mul(b)


grad_dispatcher = Autograd()

with Labeler():
    with grad_dispatcher:
        loss = simple(a, b)

print("a", a)
print("b", b)
da, db = grad_dispatcher.grad(loss, [a, b])
print("da", da)
print("db", db)

Autograd v5: torch.Size([4]) = v3 + v4
Autograd v6: torch.Size([4]) = v5 * v4
a tensor([0.4963, 0.7682, 0.0885, 0.1320])
b tensor([0.3074, 0.6341, 0.4901, 0.8964])
-- Autograd dv6 -------
------------------------
da tensor([0.3074, 0.6341, 0.4901, 0.8964])
db tensor([1.1111, 2.0364, 1.0687, 1.9249])


To compute higher order gradients, we have two options.  First,
we can do traditional PyTorch style higher order differentiation
with `create_graph=True`, writing the backpropagation computations directly
into the tape so they can be further differentiated over.  This is also
what the original Simple Autograd implementation does.

In [ ]:
def run_gradients(d1, d2):
    with Labeler():
        with d1:
            with d2 if d1 != d2 else contextlib.nullcontext():
                # our first loss
                L0 = simple(a, b).sum()

    # compute derivatives of our inputs
    dL0_da, dL0_db = d2.grad(L0, [a, b])
    print(dL0_da)

    # In real code, how would we switch from executing from d2 to d1?
    # In functorch, the d2 dispatch calls would happen in the inside of
    # a higher-order grad() or vjp() call; when we exit from this call, all
    # of the involved tensors are unwrapped.

    # now lets compute the L2 norm of our derivatives
    with d1.restore():
        L1 = torch.sum(torch.add(dL0_da.mul(dL0_da), dL0_db.mul(dL0_db)))

    # and take the gradient of that.
    # notice there are two losses involved1.
    return d1.grad(L1, [a, b])

grad_dispatcher = Autograd(create_graph=True)
da, db = run_gradients(grad_dispatcher, grad_dispatcher)

print("da", da)
print("db", db)

Autograd v13: torch.Size([4]) = v3 + v4
Autograd v14: torch.Size([4]) = v13 * v4
Autograd v15: torch.Size([]) = v14.sum(dim=None)
-- Autograd dv15 -------
Autograd v17: torch.Size([1]) = v16.unsqueeze(dim=0)
torch.Size([1])
torch.Size([4])
1
1
Autograd v18: torch.Size([4]) = v17.expand([4])
Autograd v19: torch.Size([4]) = v18 * v4
Autograd v20: torch.Size([4]) = v18 * v13
Autograd v21: torch.Size([4]) = v20 + v19
------------------------
tensor([0.3074, 0.6341, 0.4901, 0.8964])
Autograd v22: torch.Size([4]) = v19 * v19
Autograd v23: torch.Size([4]) = v21 * v21
Autograd v24: torch.Size([4]) = v22 + v23
Autograd v25: torch.Size([]) = v24.sum(dim=None)
-- Autograd dv25 -------
Autograd v27: torch.Size([1]) = v26.unsqueeze(dim=0)
torch.Size([1])
torch.Size([4])
1
1
Autograd v28: torch.Size([4]) = v27.expand([4])
Autograd v29: torch.Size([4]) = v28 * v21
Autograd v30: torch.Size([4]) = v28 * v21
Autograd v31: torch.Size([4]) = v29 + v30
Autograd v32: torch.Size([4]) = v28 * v19
Autograd v33

Our second option is to follow functorch's implementation strategy, which
is to stack two Autograd dispatchers on top of each other.  Here, it is
not necessary to `create_graph=True`, because when the backpropagator forwards
to the inner dispatcher, it will record those operations on the tape too.
But if you look at the output, you will notice something very interesting:
the first portion of the tape is exactly replicated between Autograd1 and
Autograd2: we're duplicating the tape in this case!  So PyTorch's default
implementation of backwards is more efficient, because it avoids having to
record the tape twice (although this doesn't matter too much, because the
saved tensors themselves can be shared between the two tapes, so it is just
the operator graph that is duplicated).

This is our first example of using two dispatchers.  While we are
performing the inner grad, we perform our operations on the outer
dispatcher `d2`; after we are done with the inner grad we switch to
`d1` by restoring `d1` without `d2`.  Intuitively, this corresponds from
passing out of the inner `grad` call to the outer `grad` call.

In [ ]:
d1 = Autograd(create_graph=False)
d2 = Autograd(create_graph=False)

da, db = run_gradients(d2, d1)
print("da", da)
print("db", db)

Autograd v45: torch.Size([4]) = v3 + v4
Autograd v45: torch.Size([4]) = v3 + v4
Autograd v46: torch.Size([4]) = v45 * v4
Autograd v46: torch.Size([4]) = v45 * v4
Autograd v47: torch.Size([]) = v46.sum(dim=None)
Autograd v47: torch.Size([]) = v46.sum(dim=None)
-- Autograd dv47 -------
Autograd v49: torch.Size([1]) = v48.unsqueeze(dim=0)
torch.Size([1])
torch.Size([4])
1
1
Autograd v50: torch.Size([4]) = v49.expand([4])
Autograd v51: torch.Size([4]) = v50 * v4
Autograd v52: torch.Size([4]) = v50 * v45
Autograd v53: torch.Size([4]) = v52 + v51
------------------------
tensor([0.3074, 0.6341, 0.4901, 0.8964])
Autograd v54: torch.Size([4]) = v51 * v51
Autograd v55: torch.Size([4]) = v53 * v53
Autograd v56: torch.Size([4]) = v54 + v55
Autograd v57: torch.Size([]) = v56.sum(dim=None)
-- Autograd dv57 -------
torch.Size([1])
torch.Size([4])
------------------------
da tensor([2.2222, 4.0728, 2.1373, 3.8498])
db tensor([5.0593, 9.4137, 5.2548, 9.4926])


Under what situations might it be profitable to keep the two tapes separate?
One guess we might have is if there is another functional transformation
wedged between the two autograd transformations.  We would then expect the
backwards formula we save to be different between the two tapes.  To do this, I
first need to implement batched tensors.

One unusual thing about this implementation is that we do not need to wrap
tensors to change their sizes; instead, we just override the meaning of
size() on the dispatcher to hide batch dimensions.  These calls are not
sent to the Pytorch dispatcher, so we need to explicitly call Dispatcher.size.

One case we do not
exercise in this example is implicit broadcasting when you combine a tensor
that is not batched with a tensor that is batched: without wrappers, a user
must explicitly lift (e.g., unsqueeze and expand) tensors they wish to
replicate across the batch dimension.  The code below will blindly attempt to
reinterpret a tensor as a batched tensor, even when it may not make sense (if
there is a size mismatch, however, you will get an assert failure).  Similarly,
once you exit a vmap region, all previously vmap'ed tensors "magically" become
unbatched.  functorch did not pursue this implementation because at the time
Tensor.size() was not virtual and thus it was not possible to override (this
will be changing soon).

In [ ]:
# This implementation of Batched only supports inserting a dimension
# at the very front
class Batched(Dispatcher):
    def __init__(self, *, length, name="Batched"):
        self.name = name
        self.length = length

    def _pad_to_size(self, lhs, rhs):
        lhs_size, rhs_size = self.inner.size(lhs), self.inner.size(rhs)
        lhs_dim, rhs_dim = len(lhs_size), len(rhs_size)
        if lhs_dim == rhs_dim:
            return (lhs, rhs)
        diff = rhs_dim - lhs_dim
        assert diff != 0  # sanity check
        new_final = rhs if diff < 0 else lhs
        for _ in range(abs(diff)):  # could be done as a reshape if we added that
            new_final = self.unsqueeze(new_final, 0)
        return (lhs, new_final) if diff < 0 else (new_final, rhs)

    def size(self, input):
        sizes = self.inner.size(input)
        print(sizes)
        assert sizes[0] == self.length
        return sizes[1:]

    def ones(self, size, **kwargs):
        return torch.ones([self.length,] + size)

    def mul(self, lhs, rhs):
        assert self.inner.size(lhs)[0] == self.length
        if not isinstance(rhs, float):
            assert self.inner.size(rhs)[0] == self.length
            lhs, rhs = self._pad_to_size(lhs, rhs)
        return self.inner.mul(lhs, rhs)

    def add(self, lhs, rhs):
        assert self.inner.size(lhs)[0] == self.length
        assert self.inner.size(rhs)[0] == self.length
        lhs, rhs = self._pad_to_size(lhs, rhs)
        return torch.add(lhs, rhs)

    def sum(self, input, dim=None):
        # offset all the summed over dimensions by one
        assert self.inner.size(input)[0] == self.length
        dim = tuple(
            i + 1 for i in sum_dims(input_dim=self.inner.dim(input) - 1, dim=dim)
        )
        return torch.sum(input, dim)

    def expand(self, input, sizes):
        # offset sizes by one
        assert self.inner.size(input)[0] == self.length
        new_sizes = [self.inner.size(input)[0]] + sizes
        return input.expand(new_sizes)

    def squeeze(self, input, dim):
        # offset dim by one
        assert self.inner.size(input)[0] == self.length
        return torch.squeeze(input, dim + 1)

    def unsqueeze(self, input, dim):
        # offset dim by one
        assert self.inner.size(input)[0] == self.length
        return torch.unsqueeze(input, dim + 1)

    def custom_vjp(self, fwd_fn, bwd_fn, *args):
        def batchify(fn):
            def new_fn(*args):
                with Batched(length=self.length, name='GeneratedBatched'):
                  return fn(*args)
            return new_fn

        # If we have Batched(Autograd(Torch()), then we would like the inner
        # dispatcher to receive a call to custom_vjp so that it preserves the
        # backward semantics. However, since this is the Batched dispatcher,
        # we want the innermost Torch dispatcher to run a batched version of fwd_fn
        # function! The way we get this to work is to create a new fwd_fn, that,
        # when executed, executes a batched version of fwd_fn.
        #
        # Same thing for the bwd_fn.
        # NB: currently simple_functorch assumes that all Tensors are batched at
        # dimension 0. I'm not sure how this logic would look like without
        # this assumption (in functorch tensors may not be batched).
        r, saved = super().custom_vjp(batchify(fwd_fn), batchify(bwd_fn), *args)
        return r, saved

    # The lift operation takes a tensor associated with some inner
    # dispatcher, and "lifts" it so that it is interpreted neutrally
    # for the outer dispatcher.  For most dispatchers this is trivial,
    # but for batched tensor it is not: given a tensor x, to interpret
    # it as x under the Batching dispatcher, we have to expand it so
    # that it is broadcasted along its first dimension.
    def lift(self, input, d):
        if d is self:
            return input
        b_input = torch.unsqueeze(input, 0)
        b_input = b_input.expand((self.length,) + self.inner.size(input))
        return self.inner.lift(b_input, d)

In [ ]:
# Our inputs are batched this time!
va, vb = label(torch.rand(2, 4)), label(torch.rand(2, 4))

def run_batched_gradients():
    # our first loss
    # we write the dimension we reduce on explicitly for clarity
    d1 = Autograd(create_graph=False)
    d3 = Autograd(create_graph=False)
    with Labeler():
        with d1:
            with Batched(length=2):
                with d3:
                    L0 = torch.sum(simple(va, vb), dim=0)

    # compute derivatives of our inputs
    dL0_da, dL0_db = d3.grad(L0, [va, vb])

    # now lets compute the L2 norm of our derivatives
    with d1.restore():
        L1 = torch.sum(torch.add(dL0_da.mul(dL0_da), dL0_db.mul(dL0_db)))

    # and take the gradient of that.
    # notice there are two losses involved1.
    dL1_da, dL1_db = d1.grad(L1, [va, vb])
    return dL1_da, dL1_db


dva, dvb = run_batched_gradients()
print("va", va)
print("vb", vb)
print("dva", dva)
print("dvb", dvb)

Autograd v449: torch.Size([2, 4]) = v447 + v448
torch.Size([2, 4])
Autograd v449: torch.Size([4]) = v447 + v448
Autograd v450: torch.Size([2, 4]) = v449 * v448
Autograd v450: torch.Size([2, 4]) = v449 * v448
torch.Size([2, 4])
Autograd v450: torch.Size([4]) = v449 * v448
Autograd v451: torch.Size([2]) = v450.sum(dim=[1])
torch.Size([2])
Autograd v451: torch.Size([]) = v450.sum(dim=[0])
-- Autograd dv451 -------
torch.Size([2, 4])
torch.Size([2, 4])
1
[0]
Autograd v453: torch.Size([2, 1]) = v452.unsqueeze(dim=1)
2
2
Autograd v454: torch.Size([2, 4]) = v453.expand([2, 4])
Autograd v455: torch.Size([2, 4]) = v454 * v448
Autograd v455: torch.Size([2, 4]) = v454 * v448
Autograd v456: torch.Size([2, 4]) = v454 * v449
Autograd v456: torch.Size([2, 4]) = v454 * v449
Autograd v457: torch.Size([2, 4]) = v456 + v455
------------------------
Autograd v458: torch.Size([2, 4]) = v455 * v455
Autograd v459: torch.Size([2, 4]) = v457 * v457
Autograd v460: torch.Size([2, 4]) = v458 + v459
Autograd v461:

To see that we have done this correctly, we could run the corresponding JAX:

```
from jax import grad, vmap
import jax.numpy as np

def simple(a, b):
  t = a + b
  return t * b

def L0(a, b):
  return np.sum(simple(a, b))

def L1(a, b):
  dL0_da, dL0_db = vmap(grad(L0, argnums=(0,1)), in_axes=0)(a, b)
  return (dL0_da * dL0_da + dL0_db * dL0_db).sum()

va = np.asarray([[0.4556, 0.6323, 0.3489, 0.4017],
        [0.0223, 0.1689, 0.2939, 0.5185]])
vb = np.asarray([[0.6977, 0.8000, 0.1610, 0.2823],
        [0.6816, 0.9152, 0.3971, 0.8742]])
dva, dvb = grad(L1, argnums=(0,1))(va, vb)
print("dva", dva)
print("dvb", dvb)
```

Looking over the output, the tapes look similar, but we can see that the sizes
and the arguments of the operations in question differ (after all, Autograd3 is
on the inside of the vmap, while Autograd1 is outside).  But it is still very
similar: we could imagine simply varying the dispatcher we use to process backwards
depending on when we are executing the tape.  In fact, this is exactly what an
initial, non-functorch implementation of PyTorch did to support per-sample
gradients.

Exercise: modify Autograd.grad to accept a dispatcher, and use that dispatcher
instead of self.backward_inner() when running propagator functions.  Then, rewrite
the above example so that it only has one level of Autograd:
Batched(Autograd(Torch(), create_graph=True)) and show you still get the same
result.

OK, so all of this dispatcher business is all nice and explicit, but
that's not what JAX/functorch's interface looks like.  How do we
bridge the gap?  Unlike with simple functorch 1.0, we don't have to
set the global mode since we're using the context managers and modes
to set that

In [ ]:
# now unnecessary :)
# DISPATCHER = Labeler()


# @contextlib.contextmanager
# def dispatcher(d):
#     global DISPATCHER
#     old_d = DISPATCHER
#     DISPATCHER = d
#     try:
#         yield
#     finally:
#         DISPATCHER = old_d

A dispatcher mode, however, is not enough.  Remember that in our
implementation of Batched, we blindly assumed that all tensors were
batched, even if this did not necessarily make sense.  If I have
`vmap(lambda bx: bx + y)(x)`, with `x: (B,X)` and `y: (X,)`, the
underlying operation should broadcast y to `(B,X)` and then do the
addition with x (bx advertises that it has size `(X,)` inside of the
vmap'd lambda).  To know this should happen, it is necessary for
us to know that y is not a batched tensor, but x is a batched tensor.
We'll resolve this with a wrapper class called FuncTensor, which
records both the underlying Tensor, as well as the Dispatcher which
this tensor is associated with.  In the above example, `bx.dispatcher`
might be `Batched(Torch())`, whereas `x.dispatcher` is `Torch()`.

So our general strategy is as follows:
  1. Every tensor is associated with a dispatcher
  2. You can lift tensors to dispatchers which wrap them (which can
     trigger some operations, like expand for Batched); this is
     implemented by `dispatcher_wraps`
  3. To perform an operation between to tensors, lift them so that
     they all have the same dispatcher, then do the operation on
     that dispatcher.

In [ ]:
# A dispatcher d1 wraps another dispatcher d2 if d2 is an ancestor of
# d1 in the tree structure.  We've defined this relation to be
# reflexive, in the same way issubclass(A, A) == True.
def dispatcher_wraps(d1, d2):
    # Treat this as a reflexive relation
    if d1 is d2 or (d1 is not None and d2 in d1.ancestors) or d2 is None:
        return True
    return False


# Given a list of arguments, lift them all up to a common dispatcher
# level, returning that dispatcher as well as the lifted arguments.
# Note that the current dispatcher is also accounted for by getting the current
# mode! In autodidax, this is `find_top_trace`.
def lift_and_unwrap_args(*args):
    outermost = _get_torch_dispatch_mode()
    for a in args:
        if dispatcher_wraps(outermost, a.dispatcher):
            pass
        elif dispatcher_wraps(a.dispatcher, outermost):
            # You can make this case an error as well if you don't
            # want to support non-lexical functorch tensors
            outermost = a.dispatcher
        else:
            raise TypeError("incompatible dispatcher trees")
    return (outermost,) + tuple(a.lift(outermost).tensor for a in args)

The actual implementation of the wrapper tensor which tracks the
Dispatcher for a tensor

In [ ]:
@dataclass
class FuncTensor:
    tensor: Tensor
    dispatcher: Dispatcher

    # Lift a FuncTensor to an outer dispatcher
    def lift(self, d):
        # You can only lift to a dispatcher which wraps the dispatcher
        # this FuncTensor is associated with (not vice versa, or between
        # unrelated FuncTensors).
        assert dispatcher_wraps(d, self.dispatcher)
        return FuncTensor(d.lift(self.tensor, self.dispatcher), d)

    # The general strategy for any operation performed on a tensor, we
    # lift all the arguments so that they live on the same dispatcher
    # level, and then perform the operation on that dispatcher.  The
    # resulting tensor is tagged at whatever dispatcher we had run the
    # tensor on.
    def __mul__(self, other):
        d, self, other = lift_and_unwrap_args(self, other)
        with restore_dispatcher_or_nop(d)():
            return FuncTensor(self.mul(other), d)

    def __add__(self, other):
        d, self, other = lift_and_unwrap_args(self, other)
        with restore_dispatcher_or_nop(d)():
            return FuncTensor(self.add(other), d)

    def sum(self, dim=None):
        d, self = lift_and_unwrap_args(self)
        with restore_dispatcher_or_nop(d)():
            if dim is None:
                res = self.sum()
            else:
                res = self.sum(dim)
            return FuncTensor(res, d)

    def expand(self, sizes):
        d, self = lift_and_unwrap_args(self)
        with restore_dispatcher_or_nop(d)():
            return FuncTensor(self.expand(sizes), d)

    def unsqueeze(self, dim):
        d, self = lift_and_unwrap_args(self)
        with restore_dispatcher_or_nop(d)():
            return FuncTensor(self.unsqueeze(dim), d)

    def squeeze(self, dim):
        d, self = lift_and_unwrap_args(self)
        with restore_dispatcher_or_nop(d)():
            return FuncTensor(self.squeeze(dim), d)

    def size(self):
        d, self = lift_and_unwrap_args(self)
        return d.size(self)

    def dim(self):
        d, self = lift_and_unwrap_args(self)
        return d.size(self)

    # Factory functions like ones do not have any Tensor arguments,
    # so they rely solely on the current mode
    @staticmethod
    def ones(size):
        return torch.ones(size)

Now we are ready to implement grad.  First, we need some helper
functions.

In [ ]:
# When we are done doing a vmap/grad, we need to take the results and
# lower them back to a lower dispatcher on the stack (this is always
# a no-op, in particular, in the vmap case, when we exit vmap the user
# gets to see the batched dimension again.)
def unlift(t, d):
    if isinstance(t, list):
        return [unlift(x, d) for x in t]
    elif isinstance(t, tuple):
        return tuple(unlift(x, d) for x in t)
    else:
        if t.dispatcher is d:
            return t
        return unlift(FuncTensor(t.tensor, t.dispatcher.inner), d)


# This lets us easily pick out arguments as specified by argnums
def filter_argnums(args, argnums):
    if isinstance(argnums, int):
        return (args[argnums],)
    else:
        return tuple(args[i] for i in argnums)

Now grad and vmap!

In [ ]:
# For simplicity, these functions only take tuples, not pytrees
def grad(f, argnums=0):
    @functools.wraps(f)
    def wrapped_f(*args):
        # We first lift and unwrap all of the arguments which we want
        # to pass into the function
        old_d, *args = lift_and_unwrap_args(*args)
        assert old_d == _get_torch_dispatch_mode()
        d = Autograd()
        with d:
            # We pass in the functions at the new Autograd level (they
            # were lifted to old_d, and lifting to d is a noop)
            L = f(*(FuncTensor(a, d) for a in args))
            assert L.dispatcher is d
            # Run the autograd pass, getting the grads for the inputs
            # as specified by argnums
        grads = d.grad(L.tensor, filter_argnums(args, argnums))
        # Finally, construct the grads at the lower level and return
        # them
        return [FuncTensor(r, old_d) for r in grads]

    return wrapped_f

In [ ]:
def vmap(f):
    @functools.wraps(f)
    def wrapped_f(*args):
        # cannot vmap over no arguments as this function uses the
        # arguments to determine how large the batch dimension is
        # (hypothetically, you could explicitly pass in the batch
        # size, and then use this to control factory functions;
        # JAX doesn't seem to have a knob to do this)
        assert args
        old_d, *args = lift_and_unwrap_args(*args)
        d = Batched(length=args[0].size()[0])
        for a in args:
            assert a.size()[0] == d.length
        with d:
            # Rewrap all the arguments as batched tensors, then
            # unwrap any batched tensors that escape
            return unlift(f(*(FuncTensor(a, d) for a in args)), old_d)

    return wrapped_f

Now we can rerun our example using the high level grad/vmap functions!

In [ ]:
def simple(a, b):
    t = a + b
    return t * b


def L0(a, b):
    return simple(a, b).sum()


def L1(a, b):
    dL0_da, dL0_db = vmap(grad(L0, argnums=(0, 1)))(a, b)
    return (dL0_da * dL0_da + dL0_db * dL0_db).sum()

l = Labeler()
print(va)
print(vb)
with l:
    fva = FuncTensor(va, l)
    fvb = FuncTensor(vb, l)
    dva, dvb = grad(L1, argnums=(0, 1))(fva, fvb)
print("dva", dva)
print("dvb", dvb)

tensor([[0.4194, 0.5529, 0.9527, 0.0362],
        [0.1852, 0.3734, 0.3051, 0.9320]])
tensor([[0.1759, 0.2698, 0.1507, 0.0317],
        [0.2081, 0.9298, 0.7231, 0.7423]])
Autograd v165: torch.Size([2, 4]) = v124 + v125
torch.Size([2, 4])
Autograd v165: torch.Size([4]) = v124 + v125
Autograd v166: torch.Size([2, 4]) = v165 * v125
torch.Size([2, 4])
Autograd v166: torch.Size([4]) = v165 * v125
Autograd v167: torch.Size([2]) = v166.sum(dim=[1])
torch.Size([2])
Autograd v167: torch.Size([]) = v166.sum(dim=None)
-- Autograd dv167 -------
torch.Size([2, 4])
torch.Size([2, 4])
Autograd v169: torch.Size([2, 1]) = v168.unsqueeze(dim=1)
torch.Size([2, 1])
torch.Size([1])
torch.Size([4])
2
2
Autograd v170: torch.Size([2, 4]) = v169.expand([2, 4])
Autograd v171: torch.Size([2, 4]) = v170 * v125
Autograd v172: torch.Size([2, 4]) = v170 * v165
Autograd v173: torch.Size([2, 4]) = v172 + v171
------------------------
Autograd v174: torch.Size([2, 4]) = v171 * v171
Autograd v175: torch.Size([2, 4]) = v1

Because FuncTensors are associated with the ambient dispatcher they
were created from, they are also allowed to escape from the context in
which they were defined, allowing for non-lexical, imperative
transform API.  For example, batching over module parameters is
problematic today, but all we need to do is tweak the FuncTensor's
dispatchers appropriately and everything works out.

In [ ]:
B = 2

# this is a bug, we should be able to set inner in the constructor and have that set the ancestors correctly
base_dispatcher = Labeler()
batched_dispatcher = Batched(length=B)
with base_dispatcher:
    with batched_dispatcher:
        pass

PlainTensor = lambda t: FuncTensor(torch.randn(N), base_dispatcher)
BatchedTensor = lambda t: FuncTensor(t, batched_dispatcher)

class ScaleBiasModule:
    weight: FuncTensor
    bias: FuncTensor

    def __init__(self, N):
        self.weight = PlainTensor(torch.randn(N))
        self.bias = PlainTensor(torch.randn(N))

    def forward(self, input):
        return self.weight * input + self.bias


B = 2
N = 3
m = ScaleBiasModule(N)
# Ensemble weights only; input is not batched
m.weight = BatchedTensor(torch.randn(B, N))
input = PlainTensor(torch.randn(N))
output = m.forward(input)
print(
    "expect", input.tensor.unsqueeze(0) * m.weight.tensor + m.bias.tensor.unsqueeze(0)
)
print("output", output.tensor)

expect tensor([[-0.6632,  0.5145,  0.7165],
        [-0.6439,  0.5264, -0.8546]])
output tensor([[-0.6632,  0.5145,  0.7165],
        [-0.6439,  0.5264, -0.8546]])


Higher-order operations in simple functorch!

Problem: users want to define functions with custom forward and backward
passes. These functions call PyTorch operations. When we vmap over such a
function, we would like for the backward pass to be preserved.

Why is this difficult? In PyTorch today, vmap over an autograd.Function
effectively runs vmap on the forward pass of the autograd.Function.
Meanwhile, autograd records the transformed operations for backward, instead
of the custom backward pass we specified in the autograd.Function!

Solution: We're going to introduce a `custom_vjp` primitive that accepts
functions and varargs Tensor arguments and demonstrate that it resolves
the problem.

custom_vjp(fwd_fn, bwd_fn, *args) takes in two functions as arguments.
We add a little helper function so that the user is not explicitly calling
this function on the active dispatcher

In [ ]:
def custom_vjp(fwd_fn, bwd_fn, *args):
  d = _get_torch_dispatch_mode()
  return d.custom_vjp(fwd_fn, bwd_fn, *args)

For our custom function, we want f(x) = x * x, but we install a custom
backwards pass that computes 32 * x (instead of 2 * x) so we can tell
if custom_vjp is working.

In [ ]:
a = label(torch.rand(4))
va = label(torch.rand(2, 4))

def f_fwd(x):
    # Our convention is that f_fwd returns (outputs, "saved")
    return x.mul(x), x

# Our convention is that f_bwd accepts (dispatcher, gradOutputs, "saved")
def f_bwd(gradOutputs, x):
    gO, = gradOutputs
    # Should be gO * 2 * x, but we're gonna do gO * 32 * x to demonstrate things
    with no_dispatch():
        thirty_two = torch.tensor(32.)  # a hack so I don't have to override lift

    return [torch.mul(gO.mul(x), label(thirty_two, 'thirty_two'))]

In [ ]:
def run_grad():
    grad_dispatcher = Autograd()
    with Labeler():
        with grad_dispatcher:
            # Here's how to invoke custom_vjp.
            r, _ = custom_vjp(f_fwd, f_bwd, a)
            L3 = torch.sum(r)
    print(grad_dispatcher.gradient_tape)
    dL3_a, = grad_dispatcher.grad(L3, [a])

    print(dL3_a)
    print(a)
    # Check that the gradients are indeed 32 * a
    assert torch.allclose(dL3_a, 32 * a)

run_grad()

v500 = custom_vjp(f_fwd, f_bwd, v498)
None
Autograd v501: torch.Size([]) = v500.sum(dim=None)
[TapeEntry(inputs=['v498'], outputs=['v500'], propagate=<function Autograd.custom_vjp.<locals>.propagate at 0x7f6f7f223cb0>), TapeEntry(inputs=['v500'], outputs=['v501'], propagate=<function Autograd.sum.<locals>.propagate at 0x7f6f7f223c20>)]
-- Autograd dv501 -------
1
None
------------------------
tensor([ 6.7094, 27.1989, 10.2486, 29.4958])
tensor([0.2097, 0.8500, 0.3203, 0.9217])


In [ ]:
# Multiple layers of autograd
def run_gradgrad():
    grad_dispatcher_1 = Autograd()
    grad_dispatcher_2 = Autograd()
    with Labeler():
        with Logger(name="Torch"):
            with grad_dispatcher_1:
                with grad_dispatcher_2:
                    r, _ = custom_vjp(f_fwd, f_bwd, a)
                    L4 = r.sum()
    dL4_a, = grad_dispatcher_2.grad(L4, [a])

    # Evidence that d2 respected the custom_vjp's f_bwd
    assert torch.allclose(dL4_a, 32 * a)

    assert hasattr(dL4_a, 't_name')
    with grad_dispatcher_1.restore():
        dL4_a_sum = dL4_a.sum()
    ddL4_a_a, = grad_dispatcher_1.grad(dL4_a_sum, [a])

    # Evidence that d1 respected the custom_vjp's f_bwd
    assert torch.allclose(ddL4_a_a, torch.ones_like(a) * 32)

run_gradgrad()

v507, v498 = custom_vjp(f_fwd, f_bwd, v498)
None
v507 = custom_vjp(f_fwd, f_bwd, v498)
None
v507 = custom_vjp(f_fwd, f_bwd, v498)
None
  Torch v508: torch.Size([]) = v507.sum(dim=None)
Autograd v508: torch.Size([]) = v507.sum(dim=None)
Autograd v508: torch.Size([]) = v507.sum(dim=None)
  Torch v509: torch.Size([]) = ones([])
-- Autograd dv508 -------
1
None
  Torch v510: torch.Size([1]) = v509.unsqueeze(0)
Autograd v510: torch.Size([1]) = v509.unsqueeze(dim=0)
1
1
  Torch v511: torch.Size([4]) = v510.expand([4])
Autograd v511: torch.Size([4]) = v510.expand([4])
  Torch v512: torch.Size([4]) = v511 * v498
Autograd v512: torch.Size([4]) = v511 * v498
  Torch v513: torch.Size([4]) = v512 * thirty_two
Autograd v513: torch.Size([4]) = v512 * thirty_two
------------------------
  Torch v514: torch.Size([]) = v513.sum(dim=None)
Autograd v514: torch.Size([]) = v513.sum(dim=None)
  Torch v515: torch.Size([]) = ones([])
-- Autograd dv514 -------
1
None
  Torch v516: torch.Size([1]) = v515.unsque

And now, let's try that again, with grad(lambda x: vmap(f)(x).sum()).
The goal of custom_vjp is to make it so that vmap(custom_vjp) still
preserves the backward semantics.

In [ ]:
def f_fwd(x):
    return x.mul(x), x

def f_bwd(gradOutputs, x):
    gO, = gradOutputs
    # Should be gO * 2 * x, but we're gonna do gO * 32 * x to prove a point
    return [torch.mul(gO.mul(x), torch.ones(()) * 32.)]

def run_gradvmap():
    grad_dispatcher = Autograd()
    with Labeler():
        with grad_dispatcher:
            with Batched(length=2):
                r, _ = custom_vjp(f_fwd, f_bwd, va)
            L99 = r.sum()
    dL99_a, = grad_dispatcher.grad(L99, [va])

    # As you can see, d1.grad still calls f_bwd.
    # The way we got this to work is that Batched.custom_vjp
    # calls custom_vjp on its inner dispatcher.
    # Scroll up to the implementation of Batched for more details.
    assert torch.allclose(dL99_a, 32 * va)

run_gradvmap()

v561 = custom_vjp(new_fn, new_fn, v499)
None
Autograd v562: torch.Size([]) = v561.sum(dim=None)
-- Autograd dv562 -------
2
None
------------------------
